#### The Objective

The ultimate aim of of the challenge is to **predict the area of wildfires in 7 regions in Australia for February 2021** with historical data, before they have happened! 

There are three submissions:
- 1) Predict wildfires in February 2020.
- 2) Predict wildifres in 3rd and 4th week of January 2021.
- 3) Predict wildfires in February 20201.

#### 1.4 Historical Weather Forecasts

This file contains the same variables as the weather_data, but these predicted forecasts and not observations. There is an extra column `Lead time` that gives the number of days the forecast is valid for. The below example explains this:

#### Steps:
[1. Load Packages](#LoadPackages) 

[2. Descriptive Stats](#DescriptiveStats) 

[3. Evaluating for Missing Values(no missing values)](#MissingValues) 

[4. Checking for Duplicates (no duplicates)](#Duplicates) 

[5. Rearranging Table via Pivot](#PivotTable) 

[6. Evaluate Re-Arranced Parameter Columns for Missing and Duplicates](#RearrangedTable) 

[7. Weather Forecast Data Review](#DataReview) 

[8. Save out Pre-Processed "C&P_Weather" CSV File](#PreprocessedWeather) 

#### Load packages

In [1]:
# Import the necessary packages for analysis and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt, mpld3
%matplotlib inline
import json
import datetime

from shapely.geometry import Polygon, mapping
import geopandas as gpd
import folium
from folium.plugins import TimeSliderChoropleth
import seaborn as sns
import plotly.express as px

sns.set_style("whitegrid")

import warnings
warnings.filterwarnings("ignore")

In [2]:
hwforecasts = "P:\Wildfires_Australia\cfc_wildfireforecastforAustralia\COPY\H_WeatherForecasts.csv"
print("Reading file: '{}'".format(hwforecasts))
hwforecasts_df = pd.read_csv(hwforecasts, parse_dates=[1])

print("Loaded...")

hwforecasts_df.head()

Reading file: 'P:\Wildfires_Australia\cfc_wildfireforecastforAustralia\COPY\H_WeatherForecasts.csv'
Loaded...


,Date,Region,Parameter,Lead time,count()[unit: km^2],min(),max(),mean(),variance()
0,2014-01-01,NSW,RelativeHumidity,5,8.037682e+05,7.482927,85.021118,28.223569,353.620815
1,2014-01-01,NSW,SolarRadiation,5,8.037682e+05,24.865765,33.557598,31.647308,2.276068
2,2014-01-01,NSW,Temperature,5,8.037682e+05,21.243755,36.929035,30.893523,17.918553
3,2014-01-01,NSW,WindSpeed,5,8.037682e+05,1.593531,6.989559,3.958822,1.334834
4,2014-01-01,NT,RelativeHumidity,5,1.349817e+06,14.796251,73.601479,39.799856,189.805002


#### Notes:

* Renaming the columns to make more sense.
* Data type has been changed to match across all other datasets.
* No duplicates or drops.

In [3]:
# Rename columns names
hwforecasts_df.columns = ['Date', 'Region', 'Parameter', 'Lead time', 'area', 'min_forcast', 'max_forcast', 'mean_forcast', '2nd_moment_forcast']
hwforecasts_df.head()

,Date,Region,Parameter,Lead time,area,min_forcast,max_forcast,mean_forcast,2nd_moment_forcast
0,2014-01-01,NSW,RelativeHumidity,5,8.037682e+05,7.482927,85.021118,28.223569,353.620815
1,2014-01-01,NSW,SolarRadiation,5,8.037682e+05,24.865765,33.557598,31.647308,2.276068
2,2014-01-01,NSW,Temperature,5,8.037682e+05,21.243755,36.929035,30.893523,17.918553
3,2014-01-01,NSW,WindSpeed,5,8.037682e+05,1.593531,6.989559,3.958822,1.334834
4,2014-01-01,NT,RelativeHumidity,5,1.349817e+06,14.796251,73.601479,39.799856,189.805002


#### Descriptive Stats <a class="anchor" id="DescriptiveStats"></a>

In [4]:
hwforecasts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217212 entries, 0 to 217211
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Date                217212 non-null  object 
 1   Region              217212 non-null  object 
 2   Parameter           217212 non-null  object 
 3   Lead time           217212 non-null  int64  
 4   area                217212 non-null  float64
 5   min_forcast         217212 non-null  float64
 6   max_forcast         217212 non-null  float64
 7   mean_forcast        217212 non-null  float64
 8   2nd_moment_forcast  217212 non-null  float64
dtypes: float64(5), int64(1), object(3)
memory usage: 14.9+ MB


In [5]:
hwforecasts_df.shape

(217212, 9)

In [6]:
#Changing date for consistency across all datasets
hwforecasts_df['Date'] = pd.to_datetime(hwforecasts_df['Date'])
hwforecasts_df['Date'].dtype.name

'datetime64[ns]'

#### Evaluating for Missing Values <a class="anchor" id="MissingValues"></a>

In [7]:
hwforecasts_df.isna().sum()

Date                  0
Region                0
Parameter             0
Lead time             0
area                  0
min_forcast           0
max_forcast           0
mean_forcast          0
2nd_moment_forcast    0
dtype: int64

#### Checking for Duplicates <a class="anchor" id="Duplicates"></a>

In [8]:
hwforecasts_df.duplicated().sum()

0

In [9]:
hwforecasts_df.dtypes

Date                  datetime64[ns]
Region                        object
Parameter                     object
Lead time                      int64
area                         float64
min_forcast                  float64
max_forcast                  float64
mean_forcast                 float64
2nd_moment_forcast           float64
dtype: object

In [10]:
hwforecasts_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Lead time,217212.0,9.660976e+00,4.135758,5.000000,5.000000,10.000000,1.500000e+01,1.500000e+01
area,217212.0,1.101102e+06,799723.923013,65671.416905,230045.695238,979710.252773,1.736319e+06,2.542548e+06
min_forcast,217212.0,1.097838e+01,14.678897,-5.018160,0.805676,5.663420,1.582764e+01,2.540969e+02
max_forcast,217212.0,3.313941e+01,55.526747,0.000000,9.633714,23.250812,3.797200e+01,1.000139e+04
mean_forcast,217212.0,2.020531e+01,25.018023,0.000000,3.962613,15.144843,2.922472e+01,5.079335e+03
2nd_moment_forcast,217212.0,3.672121e+02,77323.513214,0.000000,1.842921,6.909953,2.977374e+01,2.373057e+07


In [11]:
print("Number of records: {}".format(len(hwforecasts_df)))
print("Number of regions: {}\n".format(len(hwforecasts_df['Region'].unique())))
print(hwforecasts_df['Region'].unique())
print(hwforecasts_df['Parameter'].unique())

Number of records: 217212
Number of regions: 7

['NSW' 'NT' 'QL' 'SA' 'TA' 'VI' 'WA']
['RelativeHumidity' 'SolarRadiation' 'Temperature' 'WindSpeed'
 'Precipitation']


#### Re-arranging Table via Pivot Function <a class="anchor" id="PivotTable"></a>

In [12]:
#rearranging Paramater values in the weather data
df_pivot = hwforecasts_df.pivot_table(values=[ 'min_forcast', 'max_forcast', 'mean_forcast', '2nd_moment_forcast'], index=['Date','Region', 'area'], columns=['Parameter'])
df_pivot

2nd_moment_forcast                   \
Parameter                           Precipitation RelativeHumidity   
Date       Region area                                               
2014-01-01 NSW    8.037682e+05                NaN       353.620815   
           NT     1.349817e+06                NaN       189.805002   
           QL     1.736319e+06                NaN       470.624907   
           SA     9.797103e+05                NaN       275.718715   
           TA     6.567142e+04                NaN        12.570180   
...                                           ...              ...   
2020-10-31 QL     1.736319e+06          14.986532       380.939464   
           SA     9.797103e+05           0.592746       193.923947   
           TA     6.567142e+04           0.640148         8.139720   
           VI     2.300457e+05          22.698446       128.169341   
           WA     2.542548e+06           1.037259       210.176753   

                                                                      \
Parameter                      SolarRadiation Temperature  WindSpeed   
Date       Region area                                                 
2014-01-01 NSW    8.037682e+05       2.276068   17.918553   1.334834   
           NT     1.349817e+06       6.573279    6.605778   4.871655   
           QL     1.736319e+06       7.079362   22.150075   1.352935   
           SA     9.797103e+05      30.069684   15.097683  10.752086   
           TA     6.567142e+04       9.588782    2.292068   1.000159   
...                                       ...         ...        ...   
2020-10-31 QL     1.736319e+06      19.213265   11.243581   1.586587   
           SA     9.797103e+05      29.354453   10.887814   1.288522   
           TA     6.567142e+04      16.065145    1.197304   0.950032   
           VI     2.300457e+05      30.996366    7.111611   1.155491   
           WA     2.542548e+06       8.129386   20.567152   2.572708   

                                 max_forcast                                  \
Parameter                      Precipitation RelativeHumidity SolarRadiation   
Date       Region area                                                         
2014-01-01 NSW    8.037682e+05           NaN        85.021118      33.557598   
           NT     1.349817e+06           NaN        73.601479      32.766205   
           QL     1.736319e+06           NaN        83.466888      33.518051   
           SA     9.797103e+05           NaN        75.290993      33.439438   
           TA     6.567142e+04           NaN        92.093201      22.283730   
...                                      ...              ...            ...   
2020-10-31 QL     1.736319e+06     29.090143        90.172732      38.616156   
           SA     9.797103e+05      2.898387        83.288211      38.391802   
           TA     6.567142e+04      2.406794        83.861537      34.327941   
           VI     2.300457e+05     26.487278        94.456088      35.922787   
           WA     2.542548e+06      9.762932        79.242772      38.204433   

                                                       mean_forcast  \
Parameter                      Temperature  WindSpeed Precipitation   
Date       Region area                                                
2014-01-01 NSW    8.037682e+05   36.929035   6.989559           NaN   
           NT     1.349817e+06   39.907539  12.156700           NaN   
           QL     1.736319e+06   40.503181   7.026765           NaN   
           SA     9.797103e+05   38.834274  15.038714           NaN   
           TA     6.567142e+04   20.020470   8.109127           NaN   
...                                    ...        ...           ...   
2020-10-31 QL     1.736319e+06   32.907190   7.578119      1.118654   
           SA     9.797103e+05   28.583809   8.654681      0.180554   
           TA     6.567142e+04   13.944504   7.087545      0.478931   
           VI     2.300457e+05   21.131717   5.890976      1.902180 

In [13]:
hwforecasts_df.loc[(hwforecasts_df['Date'] == '2017-10-06') & (hwforecasts_df['Lead time'] == 15) &
                ((hwforecasts_df['Parameter'] == 'Precipitation')), :]

,Date,Region,Parameter,Lead time,area,min_forcast,max_forcast,mean_forcast,2nd_moment_forcast
100641,2017-10-06,NSW,Precipitation,15,7.725261e+05,250.288422,8210.618164,316.547930,2.902702e+05
100656,2017-10-06,NT,Precipitation,15,9.152510e+05,252.940247,10001.387695,1671.411758,1.077077e+07
100671,2017-10-06,QL,Precipitation,15,4.590840e+05,254.096924,10000.420898,5079.335413,2.373057e+07
100686,2017-10-06,SA,Precipitation,15,7.573878e+05,250.923462,10000.200195,1975.757703,1.319589e+07
100701,2017-10-06,TA,Precipitation,15,6.567142e+04,249.333389,255.055130,250.576076,9.660292e-01
100716,2017-10-06,VI,Precipitation,15,2.300457e+05,250.512894,258.626038,254.752662,3.580460e+00
100731,2017-10-06,WA,Precipitation,15,1.037480e+06,252.199997,10000.639648,3748.751125,2.110331e+07


Removing records where Lead time equals 15 for Precipitation forecasts on Date 2017-10-06

In [14]:
hwforecasts_df.drop(hwforecasts_df.index[(hwforecasts_df['Date'] == '2017-10-06') &
                                    (hwforecasts_df['Lead time'] == 15) &
                                    ((hwforecasts_df['Parameter'] == 'Precipitation'))], inplace=True)

In [15]:
num_rows, num_cols = hwforecasts_df.shape
print("There are total {} records in the following {} columns:\n".format(num_rows, num_cols))
print("\n".join(list(hwforecasts_df.columns)))
#7 records were removed from outliers removed above (previous record count was 217212)

There are total 217205 records in the following 9 columns:

Date
Region
Parameter
Lead time
area
min_forcast
max_forcast
mean_forcast
2nd_moment_forcast


Now to rearrange data such that the Parameter Values become columns containing values 'min', 'max', 'mean' and '2nd_moment' while keeping distinct values for 'Date', 'Region', 'Lead time' and 'area'

In [16]:
#rearranging Paramater values data
df_pivot = hwforecasts_df.pivot_table(values=['min_forcast', 'max_forcast', 'mean_forcast', '2nd_moment_forcast'], index=['Date', 'Region', 'Lead time', 'area'], columns=['Parameter'])
df_pivot

2nd_moment_forcast                   \
Parameter                                     Precipitation RelativeHumidity   
Date       Region Lead time area                                               
2014-01-01 NSW    5         8.037682e+05                NaN       353.620815   
           NT     5         1.349817e+06                NaN       189.805002   
           QL     5         1.736319e+06                NaN       470.624907   
           SA     5         9.797103e+05                NaN       275.718715   
           TA     5         6.567142e+04                NaN        12.570180   
...                                                     ...              ...   
2020-10-31 VI     10        2.300457e+05           0.226294        71.399972   
                  15        2.300457e+05           1.064420       231.553974   
           WA     5         2.542548e+06           1.479139       200.508811   
                  10        2.542548e+06           0.146198       214.113833   
                  15        2.542548e+06           1.486440       215.907616   

                                                                     \
Parameter                                SolarRadiation Temperature   
Date       Region Lead time area                                      
2014-01-01 NSW    5         8.037682e+05       2.276068   17.918553   
           NT     5         1.349817e+06       6.573279    6.605778   
           QL     5         1.736319e+06       7.079362   22.150075   
           SA     5         9.797103e+05      30.069684   15.097683   
           TA     5         6.567142e+04       9.588782    2.292068   
...                                                 ...         ...   
2020-10-31 VI     10        2.300457e+05      15.359641    7.669216   
                  15        2.300457e+05      25.315183    9.185775   
           WA     5         2.542548e+06       8.996331   14.749877   
                  10        2.542548e+06       2.844737   14.610838   
                  15        2.542548e+06      12.547091   32.340741   

                                                      max_forcast  \
Parameter                                 WindSpeed Precipitation   
Date       Region Lead time area                                    
2014-01-01 NSW    5         8.037682e+05   1.334834           NaN   
           NT     5         1.349817e+06   4.871655           NaN   
           QL     5         1.736319e+06   1.352935           NaN   
           SA     5         9.797103e+05  10.752086           NaN   
           TA     5         6.567142e+04   1.000159           NaN   
...                                             ...           ...   
2020-10-31 VI     10        2.300457e+05   1.141684      3.754553   
                  15        2.300457e+05   0.332452      5.357795   
           WA     5         2.542548e+06   2.626650      9.332361   
                  10        2.542548e+06   2.232168      2.732273   
                  15        2.542548e+06   2.859307     17.224161   

                                                                          \
Parameter                                RelativeHumidity SolarRadiation   
Date       Region Lead time area                                           
2014-01-01 NSW    5         8.037682e+05        85.021118      33.557598   
           NT     5         1.349817e+06        73.601479      32.766205   
           QL     5         1.736319e+06        83.466888      33.518051   
           SA     5         9.797103e+05        75.290993      33.439438   
           TA     5         6.567142e+04        92.093201      22.283730   
...                                                   ...            ...   
2020-10-31 VI     10        2.300457e+05        91.215370      38.469955   
                  15        2.300457e+05        93.267899      39.743999   
           WA     5         2.542548e+06        81.588310      30.319111   
                  10        2.542548e+06      

In [17]:
# Reset dataframe index
df_pivot.reset_index(inplace=True)
df_pivot.head()

Date Region Lead time          area 2nd_moment_forcast  \
Parameter                                                Precipitation   
0         2014-01-01    NSW         5  8.037682e+05                NaN   
1         2014-01-01     NT         5  1.349817e+06                NaN   
2         2014-01-01     QL         5  1.736319e+06                NaN   
3         2014-01-01     SA         5  9.797103e+05                NaN   
4         2014-01-01     TA         5  6.567142e+04                NaN   

                                                                  \
Parameter RelativeHumidity SolarRadiation Temperature  WindSpeed   
0               353.620815       2.276068   17.918553   1.334834   
1               189.805002       6.573279    6.605778   4.871655   
2               470.624907       7.079362   22.150075   1.352935   
3               275.718715      30.069684   15.097683  10.752086   
4                12.570180       9.588782    2.292068   1.000159   

            max_forcast  ...  mean_forcast                                  \
Parameter Precipitation  ... Precipitation RelativeHumidity SolarRadiation   
0                   NaN  ...           NaN        28.223569      31.647308   
1                   NaN  ...           NaN        39.799856      28.016282   
2                   NaN  ...           NaN        40.504877      30.337689   
3                   NaN  ...           NaN        26.038432      27.126219   
4                   NaN  ...           NaN        85.672655      17.549968   

                                  min_forcast                                  \
Parameter Temperature WindSpeed Precipitation RelativeHumidity SolarRadiation   
0           30.893523  3.958822           NaN         7.482927      24.865765   
1           33.379110  4.740253           NaN        14.796251      18.851019   
2           32.331884  4.178836           NaN         7.028183      17.929157   
3           34.074006  8.630796           NaN         6.606842      11.718054   
4           14.569532  4.831787           NaN        68.435989      12.340322   

                                 
Parameter Temperature WindSpeed  
0           21.243755  1.593531  
1           25.955570  1.192912  
2           22.155766  1.559428  
3           20.889954  2.831450  
4           11.401472  3.079223  

[5 rows x 24 columns]

In [18]:
# Renaming Column names
df_pivot.columns = [col[0] if not(col[1]) else '{1}_{0}'.format(*col) for col in df_pivot.columns.values]
df_pivot.head()

,Date,Region,Lead time,area,Precipitation_2nd_moment_forcast,RelativeHumidity_2nd_moment_forcast,SolarRadiation_2nd_moment_forcast,Temperature_2nd_moment_forcast,WindSpeed_2nd_moment_forcast,Precipitation_max_forcast,...,Precipitation_mean_forcast,RelativeHumidity_mean_forcast,SolarRadiation_mean_forcast,Temperature_mean_forcast,WindSpeed_mean_forcast,Precipitation_min_forcast,RelativeHumidity_min_forcast,SolarRadiation_min_forcast,Temperature_min_forcast,WindSpeed_min_forcast
0,2014-01-01,NSW,5,8.037682e+05,NaN,353.620815,2.276068,17.918553,1.334834,NaN,...,NaN,28.223569,31.647308,30.893523,3.958822,NaN,7.482927,24.865765,21.243755,1.593531
1,2014-01-01,NT,5,1.349817e+06,NaN,189.805002,6.573279,6.605778,4.871655,NaN,...,NaN,39.799856,28.016282,33.379110,4.740253,NaN,14.796251,18.851019,25.955570,1.192912
2,2014-01-01,QL,5,1.736319e+06,NaN,470.624907,7.079362,22.150075,1.352935,NaN,...,NaN,40.504877,30.337689,32.331884,4.178836,NaN,7.028183,17.929157,22.155766,1.559428
3,2014-01-01,SA,5,9.797103e+05,NaN,275.718715,30.069684,15.097683,10.752086,NaN,...,NaN,26.038432,27.126219,34.074006,8.630796,NaN,6.606842,11.718054,20.889954,2.831450
4,2014-01-01,TA,5,6.567142e+04,NaN,12.570180,9.588782,2.292068,1.000159,NaN,...,NaN,85.672655,17.549968,14.569532,4.831787,NaN,68.435989,12.340322,11.401472,3.079223


In [19]:
# Rearranging Data and column
params = df_pivot.columns.tolist()[4:]
params.sort()
forecasts_data = df_pivot[df_pivot.columns.tolist()[:4] + params].copy()
forecasts_data.head()

,Date,Region,Lead time,area,Precipitation_2nd_moment_forcast,Precipitation_max_forcast,Precipitation_mean_forcast,Precipitation_min_forcast,RelativeHumidity_2nd_moment_forcast,RelativeHumidity_max_forcast,...,SolarRadiation_mean_forcast,SolarRadiation_min_forcast,Temperature_2nd_moment_forcast,Temperature_max_forcast,Temperature_mean_forcast,Temperature_min_forcast,WindSpeed_2nd_moment_forcast,WindSpeed_max_forcast,WindSpeed_mean_forcast,WindSpeed_min_forcast
0,2014-01-01,NSW,5,8.037682e+05,NaN,NaN,NaN,NaN,353.620815,85.021118,...,31.647308,24.865765,17.918553,36.929035,30.893523,21.243755,1.334834,6.989559,3.958822,1.593531
1,2014-01-01,NT,5,1.349817e+06,NaN,NaN,NaN,NaN,189.805002,73.601479,...,28.016282,18.851019,6.605778,39.907539,33.379110,25.955570,4.871655,12.156700,4.740253,1.192912
2,2014-01-01,QL,5,1.736319e+06,NaN,NaN,NaN,NaN,470.624907,83.466888,...,30.337689,17.929157,22.150075,40.503181,32.331884,22.155766,1.352935,7.026765,4.178836,1.559428
3,2014-01-01,SA,5,9.797103e+05,NaN,NaN,NaN,NaN,275.718715,75.290993,...,27.126219,11.718054,15.097683,38.834274,34.074006,20.889954,10.752086,15.038714,8.630796,2.831450
4,2014-01-01,TA,5,6.567142e+04,NaN,NaN,NaN,NaN,12.570180,92.093201,...,17.549968,12.340322,2.292068,20.020470,14.569532,11.401472,1.000159,8.109127,4.831787,3.079223


In [20]:
num_rows, num_cols = forecasts_data.shape
print("There are total {} records in the following {} columns:\n".format(num_rows, num_cols))
print("\n".join(list(forecasts_data.columns)))

There are total 44620 records in the following 24 columns:

Date
Region
Lead time
area
Precipitation_2nd_moment_forcast
Precipitation_max_forcast
Precipitation_mean_forcast
Precipitation_min_forcast
RelativeHumidity_2nd_moment_forcast
RelativeHumidity_max_forcast
RelativeHumidity_mean_forcast
RelativeHumidity_min_forcast
SolarRadiation_2nd_moment_forcast
SolarRadiation_max_forcast
SolarRadiation_mean_forcast
SolarRadiation_min_forcast
Temperature_2nd_moment_forcast
Temperature_max_forcast
Temperature_mean_forcast
Temperature_min_forcast
WindSpeed_2nd_moment_forcast
WindSpeed_max_forcast
WindSpeed_mean_forcast
WindSpeed_min_forcast


#### Evaluate Re-Arranged Paramater Columns for Missing and Duplicates <a class="anchor" id="RearrangedTable"></a>

In [21]:
forecasts_data.isna().sum()

Date                                      0
Region                                    0
Lead time                                 0
area                                      0
Precipitation_2nd_moment_forcast       4692
Precipitation_max_forcast              4692
Precipitation_mean_forcast             4692
Precipitation_min_forcast              4692
RelativeHumidity_2nd_moment_forcast     279
RelativeHumidity_max_forcast            279
RelativeHumidity_mean_forcast           279
RelativeHumidity_min_forcast            279
SolarRadiation_2nd_moment_forcast       328
SolarRadiation_max_forcast              328
SolarRadiation_mean_forcast             328
SolarRadiation_min_forcast              328
Temperature_2nd_moment_forcast          275
Temperature_max_forcast                 275
Temperature_mean_forcast                275
Temperature_min_forcast                 275
WindSpeed_2nd_moment_forcast            321
WindSpeed_max_forcast                   321
WindSpeed_mean_forcast          

Cross checking NULL values in the arranged data. Let's pick RelativeHumidity_mean_forcast column

In [22]:
forecasts_data.loc[forecasts_data['RelativeHumidity_mean_forcast'].isna(), :]

,Date,Region,Lead time,area,Precipitation_2nd_moment_forcast,Precipitation_max_forcast,Precipitation_mean_forcast,Precipitation_min_forcast,RelativeHumidity_2nd_moment_forcast,RelativeHumidity_max_forcast,...,SolarRadiation_mean_forcast,SolarRadiation_min_forcast,Temperature_2nd_moment_forcast,Temperature_max_forcast,Temperature_mean_forcast,Temperature_min_forcast,WindSpeed_2nd_moment_forcast,WindSpeed_max_forcast,WindSpeed_mean_forcast,WindSpeed_min_forcast
560,2014-03-23,NSW,5,8.037682e+05,NaN,NaN,NaN,NaN,NaN,NaN,...,19.355496,7.199873,13.017521,28.743320,21.209778,11.813959,3.114025,7.779472,3.270816,0.703011
561,2014-03-23,NT,5,1.349817e+06,NaN,NaN,NaN,NaN,NaN,NaN,...,24.110669,17.686712,3.612621,33.679714,30.377331,25.807884,1.862877,7.517807,5.187270,2.139800
562,2014-03-23,QL,5,1.736319e+06,NaN,NaN,NaN,NaN,NaN,NaN,...,24.263057,9.167148,11.560332,33.592918,27.291569,17.680374,1.508545,7.050274,3.630924,0.893402
563,2014-03-23,SA,5,9.797103e+05,NaN,NaN,NaN,NaN,NaN,NaN,...,21.799047,10.947695,21.358818,32.686142,22.979985,14.914130,1.418101,7.940142,6.061781,1.961252
564,2014-03-23,TA,5,6.567142e+04,NaN,NaN,NaN,NaN,NaN,NaN,...,9.425926,2.684387,5.340168,16.724670,10.503687,6.034809,2.712482,11.102831,4.651797,2.021668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43809,2020-09-23,QL,15,1.736319e+06,0.013461,1.299296,0.027631,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43812,2020-09-23,SA,15,9.797103e+05,0.000949,0.578502,0.003454,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43815,2020-09-23,TA,15,6.567142e+04,0.232065,1.476720,0.434436,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43818,2020-09-23,VI,15,2.300457e+05,0.009802,0.551880,0.075300,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see that RelativeHumidity_mean_forcast has NULL values. Picking Date 2014-03-23 and verify in Original Data.

In [23]:
hwforecasts_df.loc[hwforecasts_df['Date'] == '2014-03-23', :]

,Date,Region,Parameter,Lead time,area,min_forcast,max_forcast,mean_forcast,2nd_moment_forcast
2240,2014-03-23,NSW,SolarRadiation,5,8.037682e+05,7.199873,24.159389,19.355496,14.676007
2241,2014-03-23,NSW,Temperature,5,8.037682e+05,11.813959,28.743320,21.209778,13.017521
2242,2014-03-23,NSW,WindSpeed,5,8.037682e+05,0.703011,7.779472,3.270816,3.114025
2243,2014-03-23,NT,SolarRadiation,5,1.349817e+06,17.686712,26.653238,24.110669,3.156213
2244,2014-03-23,NT,Temperature,5,1.349817e+06,25.807884,33.679714,30.377331,3.612621
2245,2014-03-23,NT,WindSpeed,5,1.349817e+06,2.139800,7.517807,5.187270,1.862877
2246,2014-03-23,QL,SolarRadiation,5,1.736319e+06,9.167148,26.346916,24.263057,3.060256
2247,2014-03-23,QL,Temperature,5,1.736319e+06,17.680374,33.592918,27.291569,11.560332
2248,2014-03-23,QL,WindSpeed,5,1.736319e+06,0.893402,7.050274,3.630924,1.508545
2249,2014-03-23,SA,SolarRadiation,5,9.797103e+05,10.947695,24.474842,21.799047,6.550752


We can see that there are no forcasts for RelativeHumidity and Precipitation on 2014-03-23 in all regions. Hence, these values below, are NULL values because they have no readings in original data. Okay now to fill in all null values with zeros in data.

In [24]:
forecasts_data = forecasts_data.fillna(0).copy()
forecasts_data.head()

,Date,Region,Lead time,area,Precipitation_2nd_moment_forcast,Precipitation_max_forcast,Precipitation_mean_forcast,Precipitation_min_forcast,RelativeHumidity_2nd_moment_forcast,RelativeHumidity_max_forcast,...,SolarRadiation_mean_forcast,SolarRadiation_min_forcast,Temperature_2nd_moment_forcast,Temperature_max_forcast,Temperature_mean_forcast,Temperature_min_forcast,WindSpeed_2nd_moment_forcast,WindSpeed_max_forcast,WindSpeed_mean_forcast,WindSpeed_min_forcast
0,2014-01-01,NSW,5,8.037682e+05,0.0,0.0,0.0,0.0,353.620815,85.021118,...,31.647308,24.865765,17.918553,36.929035,30.893523,21.243755,1.334834,6.989559,3.958822,1.593531
1,2014-01-01,NT,5,1.349817e+06,0.0,0.0,0.0,0.0,189.805002,73.601479,...,28.016282,18.851019,6.605778,39.907539,33.379110,25.955570,4.871655,12.156700,4.740253,1.192912
2,2014-01-01,QL,5,1.736319e+06,0.0,0.0,0.0,0.0,470.624907,83.466888,...,30.337689,17.929157,22.150075,40.503181,32.331884,22.155766,1.352935,7.026765,4.178836,1.559428
3,2014-01-01,SA,5,9.797103e+05,0.0,0.0,0.0,0.0,275.718715,75.290993,...,27.126219,11.718054,15.097683,38.834274,34.074006,20.889954,10.752086,15.038714,8.630796,2.831450
4,2014-01-01,TA,5,6.567142e+04,0.0,0.0,0.0,0.0,12.570180,92.093201,...,17.549968,12.340322,2.292068,20.020470,14.569532,11.401472,1.000159,8.109127,4.831787,3.079223


In [25]:
forecasts_data.isna().sum()

Date                                   0
Region                                 0
Lead time                              0
area                                   0
Precipitation_2nd_moment_forcast       0
Precipitation_max_forcast              0
Precipitation_mean_forcast             0
Precipitation_min_forcast              0
RelativeHumidity_2nd_moment_forcast    0
RelativeHumidity_max_forcast           0
RelativeHumidity_mean_forcast          0
RelativeHumidity_min_forcast           0
SolarRadiation_2nd_moment_forcast      0
SolarRadiation_max_forcast             0
SolarRadiation_mean_forcast            0
SolarRadiation_min_forcast             0
Temperature_2nd_moment_forcast         0
Temperature_max_forcast                0
Temperature_mean_forcast               0
Temperature_min_forcast                0
WindSpeed_2nd_moment_forcast           0
WindSpeed_max_forcast                  0
WindSpeed_mean_forcast                 0
WindSpeed_min_forcast                  0
dtype: int64

#### Weather Forecast Data Review <a class="anchor" id="DataReview"></a>

In [26]:
forecasts_data.dtypes

Date                                   datetime64[ns]
Region                                         object
Lead time                                       int64
area                                          float64
Precipitation_2nd_moment_forcast              float64
Precipitation_max_forcast                     float64
Precipitation_mean_forcast                    float64
Precipitation_min_forcast                     float64
RelativeHumidity_2nd_moment_forcast           float64
RelativeHumidity_max_forcast                  float64
RelativeHumidity_mean_forcast                 float64
RelativeHumidity_min_forcast                  float64
SolarRadiation_2nd_moment_forcast             float64
SolarRadiation_max_forcast                    float64
SolarRadiation_mean_forcast                   float64
SolarRadiation_min_forcast                    float64
Temperature_2nd_moment_forcast                float64
Temperature_max_forcast                       float64
Temperature_mean_forcast    

In [27]:
# frequencies for  Region column
forecasts_data.pivot_table(index= ['Region'], aggfunc='size')

Region
NSW    6373
NT     6376
QL     6376
SA     6373
TA     6373
VI     6373
WA     6376
dtype: int64

#### Saving out the final forecast_data CSV File

In [28]:
final_file = "C&P_Forecasts.csv"
print("Saving file: '{}'".format(final_file))
forecasts_data.to_csv(final_file, index=False, encoding='utf-8')
print("File Saved...")

Saving file: 'C&P_Forecasts.csv'
File Saved...


In [29]:
# check DataFrame exported
df = pd.read_csv("P:\Wildfires_Australia\cfc_wildfireforecastforAustralia\COPY\C&P_Forecasts.csv")
df['Date'] = pd.to_datetime(df['Date'])

In [30]:
df.head()

,Date,Region,Lead time,area,Precipitation_2nd_moment_forcast,Precipitation_max_forcast,Precipitation_mean_forcast,Precipitation_min_forcast,RelativeHumidity_2nd_moment_forcast,RelativeHumidity_max_forcast,...,SolarRadiation_mean_forcast,SolarRadiation_min_forcast,Temperature_2nd_moment_forcast,Temperature_max_forcast,Temperature_mean_forcast,Temperature_min_forcast,WindSpeed_2nd_moment_forcast,WindSpeed_max_forcast,WindSpeed_mean_forcast,WindSpeed_min_forcast
0,2014-01-01,NSW,5,8.037682e+05,0.0,0.0,0.0,0.0,353.620815,85.021118,...,31.647308,24.865765,17.918553,36.929035,30.893523,21.243755,1.334834,6.989559,3.958822,1.593531
1,2014-01-01,NT,5,1.349817e+06,0.0,0.0,0.0,0.0,189.805002,73.601479,...,28.016282,18.851019,6.605778,39.907539,33.379110,25.955570,4.871655,12.156700,4.740253,1.192912
2,2014-01-01,QL,5,1.736319e+06,0.0,0.0,0.0,0.0,470.624907,83.466888,...,30.337689,17.929157,22.150075,40.503181,32.331884,22.155766,1.352935,7.026765,4.178836,1.559428
3,2014-01-01,SA,5,9.797103e+05,0.0,0.0,0.0,0.0,275.718715,75.290993,...,27.126219,11.718054,15.097683,38.834274,34.074006,20.889954,10.752086,15.038714,8.630796,2.831450
4,2014-01-01,TA,5,6.567142e+04,0.0,0.0,0.0,0.0,12.570180,92.093201,...,17.549968,12.340322,2.292068,20.020470,14.569532,11.401472,1.000159,8.109127,4.831787,3.079223


In [31]:
df.shape

(44620, 24)

In [32]:
df.isna().sum()

Date                                   0
Region                                 0
Lead time                              0
area                                   0
Precipitation_2nd_moment_forcast       0
Precipitation_max_forcast              0
Precipitation_mean_forcast             0
Precipitation_min_forcast              0
RelativeHumidity_2nd_moment_forcast    0
RelativeHumidity_max_forcast           0
RelativeHumidity_mean_forcast          0
RelativeHumidity_min_forcast           0
SolarRadiation_2nd_moment_forcast      0
SolarRadiation_max_forcast             0
SolarRadiation_mean_forcast            0
SolarRadiation_min_forcast             0
Temperature_2nd_moment_forcast         0
Temperature_max_forcast                0
Temperature_mean_forcast               0
Temperature_min_forcast                0
WindSpeed_2nd_moment_forcast           0
WindSpeed_max_forcast                  0
WindSpeed_mean_forcast                 0
WindSpeed_min_forcast                  0
dtype: int64